In [4]:
import numpy as np
import matplotlib.pyplot as plt  
from hmmlearn import hmm

# Thought Process Here...

My goal here was to do parameter recovery using an HMM.

As a reminder, parameter recovery involves generating the parameters of a model using real-world data, then using our model to generate lots of simulated data. We then use that simulated data to estimate model parameters. If our simulated model parameters match our original 'real-world' model parameters, then we have successfully performed model recovery.

My intention here was to do model recovery on the Li *et al.* data. However, I'm having a hard time loading their data in and making any meaningful sense of it...

Some useful references for this:

- https://hmmlearn.readthedocs.io/en/latest/auto_examples/plot_variational_inference.html
- https://hmmlearn.readthedocs.io/en/latest/auto_examples/plot_casino.html
- https://www.cs.sjsu.edu/~stamp/RUA/HMM.pdf

In [7]:
# Make a generative model with 2 hidden states: high suspense and low suspense
gen_model = hmm.CategoricalHMM(n_components=2, random_state=99)
# At the start of the round, they're in a low suspenses state
gen_model.startprob_ = np.array([1.0, 0.0])
# Let's initialise the transition probability matrix
# Part of our experiment is to see if people are keen to reach states of high suspense, 
# so we could bear that in mind
# we'll intialise with 25% chance of staying and 75% of transitioning higher
gen_model.transmat_ = np.array([[0.25, 0.75], # Transition matrix describes the probabilities of transitioning between hidden states
                               [0.8, 0.2]])
# now we can intitialise the emissions matrix
# these are the probs of being in each hidden state given the observation
# in this initial case, say we observe 4 states: 1, 2, 3, 4 (in reality, we have like 21+ states remember)
# We can imagine that 1 and 2 are low suspense, and 3 and 4 are high suspense
gen_model.emissionprob_ = np.array([ # emission matrix describes the probabilities of being in each hidden state (rows) for each observation (cols)
    [0.4, 0.3, 0.2, 0.1], # i.e. probability we're in low suspense condition given each observation
    [0.1, 0.2, 0.3, 0.4] # probability we're in low suspense condition given each observation
])

In [38]:
rolls, gen_states = gen_model.sample(3000)

In [39]:
# Do parameter recovery on our initial model

# Split data into training and validation
x_train = rolls[:rolls.shape[0] // 2]
x_validate = rolls[rolls.shape[0] // 2:]

# Generate an initial optimal score
gen_score = gen_model.score(x_validate)
best_score = best_model = None

n_fits = 50
np.random.seed(13)
for ix in range(n_fits):
    model = hmm.CategoricalHMM(
        n_components=2, n_features=4, random_state=ix, init_params='ste'
    )
    model.fit(x_train)
    score = model.score(x_validate)
    print(f'Model #{ix}\tScore: {score}')
    if best_score is None or score > best_score:
        best_model = model
        best_score = score

print(f'Generated score: {gen_score}\nBest score: {best_score}')

Model #0	Score: -2079.2023916242306
Model #1	Score: -2085.7473541887784
Model #2	Score: -2076.5715494280234
Model #3	Score: -2081.0994465932536
Model #4	Score: -2079.1353917697265
Model #5	Score: -2074.358454441661
Model #6	Score: -2074.236949701335
Model #7	Score: -2071.8984451128363
Model #8	Score: -2080.968518959379
Model #9	Score: -2080.6361359573903
Model #10	Score: -2081.8742622767954
Model #11	Score: -2076.69108513878
Model #12	Score: -2084.6860628696963
Model #13	Score: -2080.373920441045
Model #14	Score: -2078.118810856695
Model #15	Score: -2081.0933643611447
Model #16	Score: -2080.0528072430097
Model #17	Score: -2079.8051915973524
Model #18	Score: -2078.305514372171
Model #19	Score: -2080.1039744034247
Model #20	Score: -2076.204594353516
Model #21	Score: -2078.0701733149967
Model #22	Score: -2079.4196688870416
Model #23	Score: -2078.9957554343227
Model #24	Score: -2072.616659093793
Model #25	Score: -2075.338893905001
Model #26	Score: -2079.955695350764
Model #27	Score: -2078.

In [40]:
states = best_model.predict(rolls)

In [42]:
print(f'Transmission Matrix Generated:\n{gen_model.transmat_.round(3)}\n\n'
      f'Transmission Matrix Recovered:\n{best_model.transmat_.round(3)}\n\n')

Transmission Matrix Generated:
[[0.25 0.75]
 [0.8  0.2 ]]

Transmission Matrix Recovered:
[[0.187 0.813]
 [0.636 0.364]]




In [44]:
test_rolls = np.array([[0, 0, 1, 1, 2, 2, 3]])
best_model.predict(test_rolls)

array([0, 1, 0, 1, 0, 1, 0], dtype=int64)

Build out HMM that learns the transition and emission probabilities from a data generation process.

Here we use 10 components, where a component is an observational state. As our observation, we'll use the difference between the upper bound and the current cumulative score.

There are 5 possible suspense categories.

We'll start with a simple deck of cards made up of [1, 2, 3, 4, 5].

In [54]:
gen_model = hmm.CategoricalHMM(n_components=5, random_state=99)
# Define the initial state, which will be P(no suspense) = 1.0
gen_model.startprob_ = np.array([1.0, 0.0, 0.0, 0.0, 0.0])
# Initialise transition probabilities - i.e. transitions between hidden states
gen_model.transmat_ = np.array([
    [0.1, 0.2, 0.2, 0.2, 0.3],
    [0.1, 0.2, 0.2, 0.2, 0.3],
    [0.1, 0.2, 0.2, 0.2, 0.3],
    [0.1, 0.2, 0.2, 0.2, 0.3],
    [0.1, 0.2, 0.2, 0.2, 0.3],
]) # Setting to uniform probabilities
# Initialise emission probabilities
# But as we previously said, observations aren't just the single card we draw, but the 2 cards per draw and our cumulative sum
gen_model.emissionprob_ = np.array([
    [], # If I'm in suspense state 1, then these are the probabilities of being in each observable state
    [], # If I'm in suspense state 2, ...
    [], # If I'm in suspense state 3, ...
    [], # If I'm in suspense state 4, ...
    [], # If I'm in suspense state 5, ...
])

# I think there's something here we could do where we use the Ely model to fill out the emission probabilities, 
# but I'm not sure why or how

In [ ]:
np.random.dirichlet()

In [55]:
rolls, gen_states = gen_model.sample(5000)

ValueError: emissionprob_ rows must sum to 1 (got row sums of [20 20 20 20 20])

In [46]:
rolls, gen_states

(array([[4],
        [0],
        [5],
        ...,
        [7],
        [2],
        [9]], dtype=int64),
 array([0, 4, 4, ..., 2, 1, 0]))

In [47]:
# Do parameter recovery on our initial model

# Split data into training and validation
x_train = rolls[:rolls.shape[0] // 2]
x_validate = rolls[rolls.shape[0] // 2:]

# Generate an initial optimal score
gen_score = gen_model.score(x_validate)
best_score = best_model = None

n_fits = 50
np.random.seed(13)
for ix in range(n_fits):
    model = hmm.CategoricalHMM(
        n_components=5, n_features=10, random_state=ix, init_params='ste'
    )
    model.fit(x_train)
    score = model.score(x_validate)
    print(f'Model #{ix}\tScore: {score}')
    if best_score is None or score > best_score:
        best_model = model
        best_score = score

print(f'Generated score: {gen_score}\nBest score: {best_score}')

Model #0	Score: -5759.794395951346
Model #1	Score: -5771.450658093483
Model #2	Score: -5765.021981520111
Model #3	Score: -5766.8158882599255
Model #4	Score: -5762.734601296893
Model #5	Score: -5759.905162126905
Model #6	Score: -5768.268424751389
Model #7	Score: -5760.90241431093
Model #8	Score: -5767.463672924933
Model #9	Score: -5764.193729068568
Model #10	Score: -5768.161153955828
Model #11	Score: -5774.522961011975
Model #12	Score: -5767.777258875839
Model #13	Score: -5762.003227996799
Model #14	Score: -5773.049015994076
Model #15	Score: -5766.999935855021
Model #16	Score: -5764.46306415079
Model #17	Score: -5758.692333916654
Model #18	Score: -5763.914937322704
Model #19	Score: -5761.477299642822
Model #20	Score: -5773.778972990429
Model #21	Score: -5762.12018562717
Model #22	Score: -5764.46267851638
Model #23	Score: -5762.355782255059
Model #24	Score: -5762.166076045307
Model #25	Score: -5765.850250387814
Model #26	Score: -5766.094441356654
Model #27	Score: -5779.692454713297
Model

In [48]:
states = best_model.predict(rolls)

In [49]:
print(f'Transmission Matrix Generated:\n{gen_model.transmat_.round(3)}\n\n'
      f'Transmission Matrix Recovered:\n{best_model.transmat_.round(3)}\n\n')

Transmission Matrix Generated:
[[0.2 0.2 0.2 0.2 0.2]
 [0.2 0.2 0.2 0.2 0.2]
 [0.2 0.2 0.2 0.2 0.2]
 [0.2 0.2 0.2 0.2 0.2]
 [0.2 0.2 0.2 0.2 0.2]]

Transmission Matrix Recovered:
[[0.926 0.056 0.    0.014 0.004]
 [0.1   0.041 0.049 0.001 0.809]
 [0.001 0.045 0.034 0.032 0.888]
 [0.361 0.498 0.119 0.006 0.015]
 [0.001 0.    0.196 0.002 0.801]]


